In [1]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import math
from opencage.geocoder import OpenCageGeocode
from clean import *
import json

In [2]:
df = pd.read_csv('data/profile_data.csv')

In [3]:
df = clean(df)

CLEANING DATA
...
FOUND 56783 ENTRIES
REMOVED 0 EMPTY ENTRIES
REMOVED 0 DUPLICATED ENTRIES


In [4]:
stats(df)

CALCULATING STATS
...
FOUND 56783 ENTRIES
56335 ENTRIES HAVE AGE DATA (99.2%)
19773 ENTRIES HAVE COLLEGE DATA (34.8%)
14456 ENTRIES HAVE CITY DATA (25.5%)
14540 ENTRIES HAVE JOB DATA (25.6%)
10153 ENTRIES HAVE GENDER DATA (17.9%)
55631 ENTRIES HAVE DISTANCE DATA (98.0%)
42256 ENTRIES HAVE DETAIL DATA (74.4%)
15078 ENTRIES HAVE ANTHEM DATA (26.6%)
----
AVERAGE AGE IN DATA SET: 24
NUMBER OF UNIQUE COLLEGES: 7793
NUMBER OF UNIQUE JOBS: 9530
NUMBER OF UNIQUE CITIES: 2793


In [5]:
df = fill_missing_cities(df)

FOUND 42327 MISSING CITY VALUES


/Users/matthewhwang/projects/tinderbot/clean.py:106: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  KNN_City = KNeighborsClassifier(n_neighbors=3).fit(X,y)


ValueError: ("Input contains NaN, infinity or a value too large for dtype('float64').", 'occurred at index 5')

In [132]:
df.city.isna().sum()

42327

In [167]:
df = df[-df.city.isna()]

In [168]:
df = df[-df.distance.isna()]

In [169]:
X = np.array(list(df.distance)).reshape(-1,1)
y = np.array(list(df.city)).reshape(-1,1)

In [170]:
KNN_City = KNeighborsClassifier(n_neighbors=10).fit(X,y)

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().



In [171]:
KNN_City.predict(np.array([[1981.0]]))[0]

'Tallahassee'

In [49]:
import plotly.express as px

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.

/Users/matthewhwang/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning:

calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.



In [42]:
eighteen = list(df[df.distance==18.0].city)

In [46]:
bootstrap_list = []

In [47]:
bootstrap = np.random.choice(eighteen, size=1000, replace=True)

In [52]:
test = pd.DataFrame(bootstrap)

In [56]:
test.columns = ['city']

In [90]:
filtered = df[-df.city.isna()].copy()
filtered = filtered[-filtered.distance.isna()]
city_list = {}

In [123]:
city_list[1981.0]

KeyError: 1981.0

In [95]:
for each in filtered.distance.unique():
    city_list[each] = list(filtered[filtered.distance == each].city)
    

In [172]:
def fill_city(series,city_list,KNN_model):
    if type(series.city)==float:
        try:
            city = np.random.choice(city_list[series.distance],1, replace=True)[0]
        except:
            city = KNN_model.predict(np.array([[series.distance]]))[0]
    else:
        city = series.city
    return city

In [137]:
for _,row in df.iterrows():
    row.city = fill_city(row)

In [174]:
df.city = df.apply(lambda x: fill_city(x,city_list,KNN_City), axis =1)

In [175]:
df.city.isna().sum()

0

In [177]:
df.head(10)

,name,age,college,job,city,gender,distance,details,anthem,profile_pic_urls
0,Gianna,19.0,NaN,NaN,Barnegat,NaN,2430.0,ig- _.giaannaa_\r\nsc- gianna8378,"('Slide (Remix) (feat. Pop Smoke, A Boogie Wit...",['https://images-ssl.gotinder.com/5dc0814dfc09...
2,Britanny,19.0,Santa Ana College,Boys and girls club,Garden Grove,NaN,19.0,Get to know me ☺️ Never been to Disneyland 😅 S...,NaN,['https://images-ssl.gotinder.com/5cace3ad9b7a...
3,Kayla,28.0,School,NaN,Los Angeles,Woman,18.0,"I am a creative person that loves simple, inno...",NaN,['https://images-ssl.gotinder.com/5e90cb67da1c...
4,Kimberly,18.0,"California State University, Los Angeles",NaN,Cudahy,Woman,15.0,Just for fun\r\nI’m 5’2,NaN,['https://images-ssl.gotinder.com/5e0ffa53349f...
6,Natalia,23.0,NaN,NaN,Monterey Park,NaN,9.0,"Using Tinder often & I love it, but I post mos...",NaN,['https://images-ssl.gotinder.com/5e90cebbda1c...
7,Ariana,18.0,Stockton University,NaN,Maplewood,NaN,2425.0,"I’m funny, I can make you laugh I guess:)\r\nA...",NaN,['https://images-ssl.gotinder.com/5de1e5f4b4d4...
8,Marion,18.0,NaN,NaN,Manchester,NaN,5272.0,his(panic) 🇲🇽 🇳🇴,"('Wet Dreamz', 'J. Cole')",['https://images-ssl.gotinder.com/5e05edcd2c19...
9,София,18.0,NaN,NaN,Новороссийск,NaN,6772.0,"Ласка одной львицы лучше, чем гарем дешевых гиен",NaN,['https://images-ssl.gotinder.com/5e4c009210ba...
10,Jena,29.0,Pasadena City College,NaN,Los Angeles,Woman,15.0,"Oh, how much I do not like the type of men who...",NaN,['https://images-ssl.gotinder.com/5e90cea1cb59...
11,Zorry,18.0,NaN,NaN,Erkelenz,Woman,5644.0,what are some fun hobbies aside of eating ?,NaN,['https://images-ssl.gotinder.com/5e1509538fca...
